<a href="https://colab.research.google.com/github/gr3gor1/SMS-Text-Classifier/blob/main/sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

Read the files and create dataframes

In [ ]:
train_data = pd.read_csv(train_file_path,sep='\t',header=None)
train_data.columns=['type','text']
train_data.head()

,type,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
test_data = pd.read_csv(test_file_path,sep="\t",header=None)
test_data.columns=['type','text']
test_data.head()

,type,text
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


Create labels

In [ ]:
train_data["type"]=pd.factorize(train_data["type"])[0]
test_data["type"]=pd.factorize(test_data["type"])[0]

Get distinct pairs of data

In [ ]:
train_set = tf.data.Dataset.from_tensor_slices((train_data["text"].values,train_data["type"].values))
test_set = tf.data.Dataset.from_tensor_slices((test_data["text"].values,test_data["type"].values))

Alter data to get the correct number of dimensions

In [ ]:
batch_size = 32
buffer_size = 100
train_set = train_set.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_set = test_set.batch(batch_size).prefetch(tf.data.AUTOTUNE)

Vectorize text messages so as to find the size of vocabulary

In [ ]:
vector = tf.keras.layers.TextVectorization(
    output_mode = 'int',
    max_tokens = 1000,
    output_sequence_length=1000
)

vector.adapt(train_set.map(lambda text , label : text))

Create model

In [ ]:
model = tf.keras.Sequential([
    vector,
    tf.keras.layers.Embedding(len(vector.get_vocabulary()),64,mask_zero=True),
    tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (TextV  (None, 1000)             0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 1000, 64)          64000     
                                                                 
 lstm_6 (LSTM)               (None, 1000, 32)          12416     
                                                                 
 lstm_7 (LSTM)               (None, 64)                24832     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                      

Train and compile model

In [ ]:
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=['acc'])

history = model.fit(train_set,validation_data=test_set,epochs=10,validation_steps=10)

Epoch 1/10
131/131 [==============================] - 37s 217ms/step - loss: 0.1843 - acc: 0.9617 - val_loss: 0.1749 - val_acc: 0.9719
Epoch 2/10
131/131 [==============================] - 13s 103ms/step - loss: 0.1207 - acc: 0.9813 - val_loss: 0.1152 - val_acc: 0.9844
Epoch 3/10
131/131 [==============================] - 13s 97ms/step - loss: 0.0919 - acc: 0.9902 - val_loss: 0.1163 - val_acc: 0.9875
Epoch 4/10
131/131 [==============================] - 13s 97ms/step - loss: 0.0835 - acc: 0.9928 - val_loss: 0.1714 - val_acc: 0.9750
Epoch 5/10
131/131 [==============================] - 13s 99ms/step - loss: 0.0701 - acc: 0.9940 - val_loss: 0.1547 - val_acc: 0.9844
Epoch 6/10
131/131 [==============================] - 13s 99ms/step - loss: 0.0642 - acc: 0.9940 - val_loss: 0.2432 - val_acc: 0.9844
Epoch 7/10
131/131 [==============================] - 12s 91ms/step - loss: 0.0545 - acc: 0.9957 - val_loss: 0.2054 - val_acc: 0.9812
Epoch 8/10
131/131 [==============================] - 11s 88

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  prediction = model.predict([pred_text])
  return [prediction[0][0], "ham" if prediction[0][0] <0.5 else "spam"]

pred_text = "call now FOR 10 % off"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 33ms/step
[1.3288069, 'spam']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 37ms/step
You passed the challenge. Great job!
